# Exercise 1.3 - Polling for real-time vehicle positions
data is pulled from [https://gtfs.adelaidemetro.com.au/v1/realtime/vehicle_positions](https://gtfs.adelaidemetro.com.au/v1/realtime/vehicle_positions)


In [1]:
# import required libraries
from google.transit import gtfs_realtime_pb2
import pandas as pd
import requests
from protobuf_to_dict import protobuf_to_dict
import hana_ml
from hana_ml.dataframe import ConnectionContext
from hana_ml.dataframe import create_dataframe_from_pandas

In [2]:
# Connect to SAP HANA Cloud
host = '[YourHostName]'
port = 443
user = '[YourUser]'
passwd = '[YourUserPassword]'
cc= ConnectionContext(address=host, port=port, user=user, password=passwd, encrypt='true' ,sslValidateCertificate='false')

schema="TECHED_USER_000"
print('HANA version:', cc.hana_version())
print('hana-ml version:', hana_ml.__version__)
print('pandas version:', pd.__version__)

HANA version: 4.00.000.00.1663064250 (fa/CE2022.30)
hana-ml version: 2.14.22092300
pandas version: 1.2.5


In [3]:
# function that get vehicle positions and returns data as pandas dataframe
def get_locations_as_df() -> pd.DataFrame:
    feed = gtfs_realtime_pb2.FeedMessage()
    response = requests.get('https://gtfs.adelaidemetro.com.au/v1/realtime/vehicle_positions')
    feed.ParseFromString(response.content)
    dict = protobuf_to_dict(feed)
    df = pd.json_normalize(dict['entity'])
    df['vehicle.timestamp'] = pd.to_datetime(df['vehicle.timestamp'], unit='s')
    return df

In [4]:
# inspect the data
df = get_locations_as_df()
df.head(3)

,id,vehicle.trip.trip_id,vehicle.trip.start_date,vehicle.trip.schedule_relationship,vehicle.trip.route_id,vehicle.trip.direction_id,vehicle.position.latitude,vehicle.position.longitude,vehicle.position.bearing,vehicle.position.speed,vehicle.timestamp,vehicle.vehicle.id,vehicle.vehicle.label
0,V550114101,550114,20221014,0,GLNELG,0,-34.927074,138.599747,4.320000,6.8,2022-10-14 12:41:15,101,101
1,V6522671026,652267,20221014,0,H22,0,-34.924629,138.593750,87.000000,0.0,2022-10-14 12:41:07,1026,1026
2,V7260941027,726094,20221014,0,281,0,-34.920921,138.611359,261.079987,9.2,2022-10-14 12:41:10,1027,1027


In [5]:
# function that calls get_locations_as_df() and stores the data in HANA tables
def store_locations():
    df = get_locations_as_df()
    hdf_locations_history = create_dataframe_from_pandas(
        connection_context=cc,
        pandas_df=df, 
        geo_cols=[("vehicle.position.longitude", "vehicle.position.latitude")], srid=4326,
        schema=schema,
        table_name='LOC_RT_HISTORY', 
        force=False, append=True
    )
    hdf_locations = create_dataframe_from_pandas(
        connection_context=cc,
        pandas_df=df, 
        geo_cols=[("vehicle.position.longitude", "vehicle.position.latitude")], srid=4326,
        schema=schema,
        table_name='LOC_RT', 
        primary_key=['id','trip_id','route_id'],
        force=True
    )


In [8]:
# try
store_locations()

100%|██████████| 1/1 [00:00<00:00, 10.77it/s]


In [6]:
# periodically import vehicle position data
import time
while True:
  store_locations()
  time.sleep(15)


100%|██████████| 1/1 [00:00<00:00, 15.16it/s]


KeyboardInterrupt: 